# Zestaw 1


## hex do base64
### [zadanie 1](https://cryptopals.com/sets/1/challenges/1)


Polega na zamianie tekstu zakodowanego w [hex](https://www.freecodecamp.org/news/hexadecimal-number-system/) (systemie szesnastkowym) na kodowanie [base64](https://www.base64encoder.io/learn/). \
Przykład:


In [135]:
data = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"

Powinien zwrócić:\
**SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t**

In [136]:
from base64 import b64decode,b64encode
def hex_to_base64(text):
    return b64encode(bytes.fromhex(text)).decode()
def base64_to_hex(text):
    return b64decode(text.encode()).hex()
print(hex_to_base64(data))

SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t


### Zadanie 2

Napisz funkcję xorująca napisy o tej samej długości

In [144]:
def xor(first: bytes, second: bytes) -> bytes:
    result = bytearray(len(first))
    for ind,(x,y) in enumerate(list(zip(first,second))):
        result[ind] = x ^ y
    return result
example1 = bytes.fromhex("1c0111001f010100061a024b53535009181c") 
example2 = bytes.fromhex("686974207468652062756c6c277320657965")

print(xor(example1,example2).decode())

the kid don't play


### Zadanie 3

Dostaliśmy informacje, że nasz szyfrogram to tekst zxorowany z jednym znakiem:

In [138]:
ciphertext = "1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736"
ciphertext = bytes.fromhex(ciphertext)

Teraz wystarczy, że sprawdzimy nasz szyfrogram z każdym możliwym znakiem. \
By nie przeglądać możliwych tekstów jawnych ręcznie, potrzebujemy funkcji, która będzie oceniała na ile dobry jest wynik.


In [139]:
freq = { ' ' : 1, 'e' : 1.3, 'E': 1.3, 'T' : 0.91, 't' : 0.91, 'A' : 0.82, 'a' : 0.82, 'O' : 0.75, 'o': 0.75,
         'I' : 0.7, 'i': 0.7,'N' : 0.67, 'n':0.67,'S' : 0.63, 's': 0.63, 'H' : 0.61, 'h': 0.61, 'R' : 0.6,
         'r': 0.6, 'D' : 0.43, 'd':0.43,'L' : 0.4 , 'l': 0.4 ,    'C' : 0.28 , 'c': 0.28 ,'U' : 0.28 , 'u': 0.28 ,
         'M' : 0.24 , 'm': 0.24 ,'W' : 0.24 , 'w': 0.24 , 'F' : 0.22 , 'f': 0.22 ,'G' : 0.2 , 'g': 0.2 ,
         'Y' : 0.2 , 'y': 0.2, 'P' : 0.19 , 'p': 0.19, 'B' : 0.15, 'b':0.15, 'V' : 0.098, 'v' : 0.098, 'K' : 0.077,
         'k' : 0.077, 'X' : 0.015, 'J': 0.015}
alphabet = b'ABCDEFGHIJKLMNOPRSTUVWXY ' 

def getScore(text):
    score = 0
    text = text.upper()
    length = len(text)
    if length == 0:
        return 0
    for letter in alphabet:
        score += round(10*freq[chr(letter)] *(text.count(letter)/length),3)
    return round(score,2)

In [167]:
def xorSingleByte(byte: int, text: bytes) -> bytes:
    return bytes(map(lambda h: h ^ byte, text))

plaintexts = []
for character in range(65,127):
    possiblePlaintext = xorSingleByte(character, ciphertext)
    plaintexts.append((getScore(possiblePlaintext),possiblePlaintext))
plaintexts = sorted(plaintexts, reverse=True)
for ind,(_,option) in enumerate(plaintexts[:3],1):
    print(f"{ind} opcja to {option.decode()}")

1 opcja to Cooking MC's like a pound of bacon
2 opcja to iEEACDM
Yi
FCAO
K
ZE_DN
EL
HKIED
3 opcja to Ieeacdm*GI-y*fcao*k*zedn*el*hkied


Funkcja, która przyda się do nastepnej części. Zwraca jedynie jeden najbardziej prawdopodobny napis.

In [182]:
def decryptSingleByte(ciphertext: str) -> tuple:
    ciphertext = ciphertext.encode()
    plaintexts = []
    for character in range(128):
        possiblePlaintext = xorSingleByte(character, ciphertext)
        plaintexts.append((getScore(possiblePlaintext),possiblePlaintext,chr(character)))
    plaintexts = sorted(plaintexts, reverse=True)
    return plaintexts[0]

### Zadanie 4

Zadanie podobne do wcześniejszego, jednakże mamy teraz 327 szyfrogramów, a jedynie jeden z nich jest szukanym.


In [176]:
texts = []
text = ""
with open("4.txt") as file:
    lines = file.readlines()
    for line in lines:
        texts.append(bytes.fromhex(line.rstrip()))
possiblePlaintexts = []
for ciphertext in texts:
    possiblePlaintexts.append(decryptSingleByte(ciphertext))
possiblePlaintexts = sorted(possiblePlaintexts, reverse=True)

for ind,(_,option,_) in enumerate(possiblePlaintexts[:3],1):
    try:
        print(f"{ind} opcja to {option.decode()}")
    except:
        print(f"{ind} decode() nie zadziałało")

1 opcja to Now that the party is jumping

2 opcja to tHuvGUEMTViEV\sLJn\Y#EieTV
3 decode() nie zadziałało


Bez problemu odnajdujemy szukany tekst jawny!

### Zadanie 5

Celem kolejnego zadania jest zaimplementowanie xora z powtarzającym się kluczem.

In [185]:
def xorRepeatingKey(text: bytes, key: bytes) -> bytes:
    result = bytearray(len(text))
    for i, byte in enumerate(text):
        result[i] = byte ^ key[i % len(key)]
    return result

text = "Burning 'em, if you ain't quick and nimble\nI go crazy when I hear a cymbal"
expectedResult = "0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f"
assert xorRepeatingKey(text.encode(), b'ICE').hex() == expectedResult


### Zadanie 6

Tym razem nie znamy długości klucza, do jego znalezienia użyjemy tzw. Hamming distance (HD) - ilość różniącyh się bitów.

Dla poniższego przykładu HD wynosi 4, widzimy, że obliczanie HD sprowadza się do obliczenia xora obu ciągów binarnych i zliczeniu liczby 1.
> 0 1 0 1 1 0 1\
> 1 0 1 1 1 0 0




Do czego nam się to może przydać? Zauważmy, że alfabet angielski wraz z znakami interpunkcyjnymi to wartości `65-127` w ASCII. XOR dowolonych takich znaków da nam średnio mniejszy HD niż dla dowolnych bajtów `0-255`. 

In [178]:
import string

def hammingDistance(messageA: bytes, messageB: bytes):
    if (type(messageA) == str):
        messageA = messageA.encode()
    if (type(messageB) == str):
        messageB = messageB.encode()
    answer = "".join([bin(x^y)[2:] for x,y in zip(messageA,messageB)])
    return answer.count("1")

def hdASCII():
    hd = 0
    count = 0 
    for i in string.printable: 
        for j in string.printable:
            count += 1
            hd += hammingDistance(i,j)
    return hd/count
def hdAllBytes():
    hd = 0
    count = 0
    for i in range(0,255):
        for j in range(0,255):
            count += 1
            hd += hammingDistance(chr(i),chr(j))
    return hd/count
print(f"Średni HD dla ascii {round(hdASCII(),2)}\nŚredni HD dla dowolnego bajtu {round(hdAllBytes(),2)}")

Średni HD dla ascii 3.43
Średni HD dla dowolnego bajtu 4.0


Szukając długości klucza będziemy zakładać, że klucz ma długość `n` po czym będziemy xorować kolejne bloki. Dzięki temu będziemy xorować bloki z tekstem jawnym, co przy odpowiendniej długości klucza, powinno dać najmniejszy HD.

$(A \oplus K) \oplus (B \oplus K) = A \oplus B $

In [179]:
def findKeySize(ciphertext: bytes) -> int:
    averageDistances = []

    # possibles key lengths
    for keyLenght in range(2,40):
        # distances in blocks
        distances = []
        chunks = [ciphertext[i:i+keyLenght] for i in range(0,len(ciphertext),keyLenght)]
        for i in range(0,len(chunks),2):
            try:
                distance = hammingDistance(chunks[i],chunks[i+1])
                distances.append(distance/keyLenght)
            except:
                break
        result = {"avgDistance": sum(distances)/len(distances),"key":keyLenght} 
        averageDistances.append(result)
    averageDistances = sorted(averageDistances, key = lambda x: x["avgDistance"])
    possibleKeyLenght = averageDistances[0]["key"]
    return possibleKeyLenght


Znając długość klucza, możemy teraz podzielić wiadomość na bloki co `n` znaku. Ilość bloków wynosi tyle co długość klucza. Znaki w jednym bloku zostały zxorowane tym samym znakiem. 

In [189]:
def xorBruteKey(ciphertext: bytes) -> bytes:
    keyLength = findKeySize(ciphertext)
    blocks = [[] for _ in range(keyLength)]

    for index,character in enumerate(ciphertext):
        blockIndex = index % keyLength
        blocks[blockIndex].append(chr(character))
    key = ""
    for block in blocks:
        key += decryptSingleByte("".join(block))[2]
    return key.encode()
xorBruteKey(data)
    

b'Terminator X: Bring the noise'

Mając funkcje, która znajduje klucz wystarczy jedynie odszyfrować naszą wiadomość. Tekst jawny jest tekstem piosenki, wypisuje jedynie pierwsze 160 znaków.

In [199]:

with open("6.txt") as file:
    data = "".join(file.read().splitlines())
    data = b64decode(data)

key = xorBruteKey(data)
plaintext = xorRepeatingKey(data,key)
print(plaintext.decode()[:160])

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 



### Zadanie 7

Celem zadania jest użycie AES w kodzie. Dostajemy dany klucz oraz plik z szyfrogramem.


In [205]:
from Crypto.Cipher import AES

key = b"YELLOW SUBMARINE"
aes = AES.new(key,AES.MODE_ECB)

def b64_hex(text: str) -> bytes:
    return b64decode(text.encode())

with open("7.txt") as file:
    text = file.readlines()
    text = "".join(text).rstrip()
text = b64_hex(text)
text = aes.decrypt(text)
#print(text.decode())


### Zadanie 8

Teraz, gdy już potrafimy używać AES w kodzie, musimy rozróżnić, który z szyfrogramów został zaszyfrowany trybem ECB. Bloki w trybie ECB nie wpływają na siebie, jeśli więc któryś szyfrogram zawiera takie same bloki, jest to przesłanka, że został użyty tryb szyfrowania ECB.



In [206]:
# Checking if every block is unique
def unique(data: list):
    for n in data:
        if data.count(n)>1:
            return 0
    return 1

with open("8.txt") as file:
    lines = file.readlines()
data = [] 
for line in lines:
    blocks = []
    line = line.rstrip()
    for i in range(0,len(line),32):
        blocks.append(line[i:i+32])
    if len(blocks):
        data.append(blocks)
for ciphertext in data:
    if not unique(ciphertext):
        print("Ten szyfrogram został zaszyfrowany ECB!\n","".join(ciphertext))
        print(ciphertext)


Ten szyfrogram został zaszyfrowany ECB!
 d880619740a8a19b7840a8a31c810a3d08649af70dc06f4fd5d2d69c744cd283e2dd052f6b641dbf9d11b0348542bb5708649af70dc06f4fd5d2d69c744cd2839475c9dfdbc1d46597949d9c7e82bf5a08649af70dc06f4fd5d2d69c744cd28397a93eab8d6aecd566489154789a6b0308649af70dc06f4fd5d2d69c744cd283d403180c98c8f6db1f2a3f9c4040deb0ab51b29933f2c123c58386b06fba186a
['d880619740a8a19b7840a8a31c810a3d', '08649af70dc06f4fd5d2d69c744cd283', 'e2dd052f6b641dbf9d11b0348542bb57', '08649af70dc06f4fd5d2d69c744cd283', '9475c9dfdbc1d46597949d9c7e82bf5a', '08649af70dc06f4fd5d2d69c744cd283', '97a93eab8d6aecd566489154789a6b03', '08649af70dc06f4fd5d2d69c744cd283', 'd403180c98c8f6db1f2a3f9c4040deb0', 'ab51b29933f2c123c58386b06fba186a']
